In [1]:
import os

#Change working directory to /Users/albertvong/Documents/Argonne/PtychoPINN/
os.chdir('/home/av_linux/PtychoPINN')

In [2]:
import numpy as np
import importlib

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Subset

import lightning as L
import mlflow.pytorch
from mlflow import MlflowClient

from ptycho_torch.dset_loader_pt_mmap import PtychoDataset
from ptycho_torch.model import PtychoPINN

import matplotlib.pyplot as plt

from ptycho_torch.config_params import TrainingConfig, DataConfig, ModelConfig

/root/miniconda3/envs/ptychopinn_ptychodus_pytorch/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data_config_default = {
    'nphotons': 1e5,
    'N': 128,
    'C': 4,
    'K': 6,
    'n_subsample': 10,
    'grid_size': (2,2),
    'probe_dir_get': True,
}


model_config_default = {
    'intensity_scale_trainable': True,
    'max_position_jitter': 10, #Random jitter for translation (helps make model more robust)
    'n_filters_scale': 2, #Shrinking factor for channels
    'intensity_scale': 15000.0, #General intensity scale guess, this can be trainable. Needs to be float
    'object.big': True, #True if need patch reassembly
    'probe.big': True, #True if need patch reassembly
    'offset': 4,
    'loss_function': 'Poisson'
}

training_config_default = {
    'nll': True, #Negative log likelihood for loss function
    'device': 'cuda'
}

t_config, d_config, m_config = TrainingConfig(), DataConfig(), ModelConfig()

t_config.set_settings(training_config_default)
d_config.set_settings(data_config_default)
m_config.set_settings(model_config_default)

In this notebook we'll be testing the training of PtychoPINN in Pytorch.

First, we're going to generate a "fake" test dataset using data from /ptycho/datasets/Run1084_recon_postPC_

# Testing out datagen methods

In [4]:
import ptycho_torch.helper as hh
import ptycho_torch.datagen as dg
import importlib

importlib.reload(hh)

<module 'ptycho_torch.helper' from '/home/av_linux/PtychoPINN/ptycho_torch/helper.py'>

In [7]:
output = dg.simulate_from_npz('ptycho_torch/datasets/als128.npy.npz', 2000)

In [8]:
#Save output as npz file in similar location
np.savez('ptycho_torch/datasets/als128_generated.npy.npz',
         diff3d = output['diff3d'],
         xcoords = output['xcoords'],
         ycoords = output['ycoords'],
         xcoords_start = output['xcoords_start'],
         ycoords_start = output['ycoords_start'])


# Testing out tensorloader and model throughput

In [4]:
import importlib
import ptycho_torch
importlib.reload(ptycho_torch.helper)
importlib.reload(ptycho_torch.dset_loader_pt_mmap)

<module 'ptycho_torch.dset_loader_pt_mmap' from '/home/av_linux/PtychoPINN/ptycho_torch/dset_loader_pt_mmap.py'>

In [5]:
from ptycho_torch.dset_loader_pt_mmap import TensorDictDataLoader
import ptycho_torch.helper as hh


In [6]:
#Setting probes

#Need better separation of params and configs... confusing
#Could separate into 
#1. Model 
#2. Training
#3. Data


dset_raw = np.load('ptycho_torch/datasets/als128.npy.npz')

probe = dset_raw['probeGuess'][None]
probe = torch.from_numpy(probe)

# d_config.add('probes', probe)

In [13]:
test = np.load('ptycho_torch/datasets/probes/probe1.npz')
torch.from_numpy(test['probe'])

tensor([[[0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j],
         ...,
         [0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j, 0.+0.j,  ..., 0.+0.j, 0.+0.j, 0.+0.j]]])

In [7]:
test_dset = PtychoDataset('ptycho_torch/datasets/dummy_data_small', 'ptycho_torch/datasets/probes',remake_map = True)

Creating memory mapped tensor dictionary...
Memory map length: 20000
Memory map creation time: 0.0035142898559570312
Populating memory map for dataset 0
Non-diffraction memory map write time: 0.0010762214660644531
Diffraction memory map write time: 6.631332159042358


In [15]:
test_dset.data_dict['probes']

AttributeError: 'NoneType' object has no attribute 'shape'

In [8]:
#Loading dataloader with first batch

testloader = TensorDictDataLoader(test_dset, batch_size = 64)
#Grab first iter from testloader
test_iter = next(iter(testloader))
#Move test_iter to gpu

#Setting different tensors for model input
diff = test_iter[0]['images'].to('cuda')
positions = test_iter[0]['coords_relative'].to('cuda')
probe = test_iter[1].to('cuda')
scaling = test_iter[2].to('cuda')

TypeError: 'NoneType' object is not subscriptable

In [14]:
model = PtychoPINN()
model.to('cuda')

PtychoPINN(
  (autoencoder): Autoencoder(
    (encoder): Encoder(
      (blocks): ModuleList(
        (0): ConvPoolBlock(
          (conv1): Conv2d(4, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
        )
        (1): ConvPoolBlock(
          (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
        )
        (2): ConvPoolBlock(
          (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1

In [15]:
model.training = True

In [11]:
model.forward_model.probe_illumination.probe_mask.device

device(type='cuda', index=0)

In [16]:
out = model(diff.to(torch.float32), positions.to(torch.float32), probe, scaling)

torch.Size([64, 4, 128, 128]) torch.Size([64, 1, 1, 1])


In [18]:
out

tensor([3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952,
        3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952,
        3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952,
        3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952,
        3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952,
        3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952,
        3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952, 3.5952,
        3.5952], device='cuda:0', dtype=torch.float64, grad_fn=<NegBackward0>)

In [19]:
model.training = False
out_notrain = model(diff.to(torch.float32), positions.to(torch.float32), probe, scaling)

out_notrain

torch.Size([64, 4, 128, 128]) torch.Size([64, 1, 1, 1])


tensor([[[[1.1875e-05, 1.5400e-05, 2.2037e-05,  ..., 2.3045e-05,
           1.1115e-05, 2.5700e-05],
          [2.2944e-05, 1.0267e-05, 3.6161e-05,  ..., 2.0793e-05,
           2.7142e-05, 1.9291e-05],
          [2.3617e-05, 1.5591e-05, 1.7429e-05,  ..., 1.1522e-05,
           2.5235e-05, 1.8305e-05],
          ...,
          [2.5047e-05, 3.3321e-05, 2.2103e-05,  ..., 1.8326e-05,
           3.8828e-06, 1.8048e-05],
          [6.2500e-06, 1.6357e-05, 1.4472e-05,  ..., 2.6845e-05,
           2.0004e-05, 1.4832e-05],
          [1.0256e-05, 1.2655e-05, 1.6549e-05,  ..., 2.9771e-05,
           2.0666e-05, 2.5537e-05]],

         [[1.1875e-05, 1.5400e-05, 2.2037e-05,  ..., 2.3045e-05,
           1.1115e-05, 2.5700e-05],
          [2.2944e-05, 1.0267e-05, 3.6161e-05,  ..., 2.0793e-05,
           2.7142e-05, 1.9291e-05],
          [2.3617e-05, 1.5591e-05, 1.7429e-05,  ..., 1.1522e-05,
           2.5235e-05, 1.8305e-05],
          ...,
          [2.5047e-05, 3.3321e-05, 2.2103e-05,  ..., 1.8326

Basic training loop (to make sure something is happening)

In [26]:
from ptycho_torch.config_params import training_config_default

training_config_default

{'nll': True}